Notebook inspired by the great notebook from Chris Deotte

RAPIDS TfidfVectorizer http://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700.

Rapids are great but you can get also great performance with sklearn.

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances_chunked

In [ ]:
COMPUTE_CV = True

test = pd.read_csv('../input/shopee-product-matching/test.csv')

if len(test)>3: COMPUTE_CV = False
    
if COMPUTE_CV:
    train = pd.read_csv('../input/shopee-product-matching/train.csv')
    tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
    train['target'] = train.label_group.map(tmp)
    print('train shape is', train.shape )
    train.head()
    
else:
    train = pd.read_csv('../input/shopee-product-matching/test.csv')


In [ ]:
print('Computing text embeddings...')
vectorizer = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
train_embeddings = vectorizer.fit_transform(train['title'])
print('text embeddings shape',train_embeddings.shape)

In [ ]:
preds1=[]
r = 0.45
start=0
end=0

def reduce_func(D_chunk, start):
    neigh = [np.flatnonzero(d < r) for d in D_chunk]
    return neigh


for neigh in pairwise_distances_chunked(train_embeddings, reduce_func=reduce_func, metric='cosine',n_jobs=-1):
    start+=end
    end+=len(neigh)
    print(f'chuck {start} to {end}')
    for i in range(len(neigh)):
        o = train.iloc[neigh[i]].posting_id.values
        preds1.append(o)

train['preds1']=preds1

In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['preds2'] = train.image_phash.map(tmp)
train.head()

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

def combine_for_cv(row):
    x = np.concatenate([row.preds1,row.preds2])
    return np.unique(x)

def combine_for_sub(row):
    x = np.concatenate([row.preds1,row.preds2])
    return ' '.join( np.unique(x) )


if COMPUTE_CV:
    train['oof'] = train.apply(combine_for_cv,axis=1)
    train['f1'] = train.apply(getMetric('oof'),axis=1)
    print('CV Score =', train.f1.mean() )
    
    
    
train['matches'] = train.apply(combine_for_sub,axis=1)

In [ ]:
train[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head(10)